<a href="https://colab.research.google.com/github/YUPENG123/googlecolab/blob/main/perceptron/perceptron_theano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://github.com/julienr/ipynb_playground
julienr/ipynb_playground

# Implementing a simple perceptron in python - theano version

In [ ]:
import pylab as pl
import numpy as np
import matplotlib.cm as cm
import sklearn.cross_validation as skcross
%matplotlib inline

np.set_printoptions(precision=5, suppress=True)

## Very quick introduction to theano

[Theano](http://www.deeplearning.net/software/theano/) is basically a code generator that compiles mathematical expressions into computer code.

The basic idea to keep in mind is that when using Theano, you work with *symbolic* variables, like you do when you write mathematical expressions. It is only when you call *theano.function* that code
(GPU code if you target the GPU, numpy code if you target the CPU) gets generated to compute your function.

In [ ]:
# Note that this require a kernel restart to be effective
import os
os.environ['THEANO_FLAGS'] = 'device=cpu'
#os.environ['THEANO_FLAGS'] = 'device=gpu'

In [ ]:
import theano.tensor as T
import theano
import tempfile
import pydot
from IPython.display import SVG
from IPython.display import display

print 'device : ', theano.config.device

device :  cpu


In [ ]:
def show_computation_graph(fn):
    """
    Wrapper around theano.printing.pydotprint that show the computation graph
    for a given theano function
    Args:
        fn: the function for which to show the computation graph

    This requires pydot and graphviz
    """
    display(SVG(theano.printing.pydotprint(fn, return_image=True, format='svg')))

We will start with simple theano expressions

### a*x + b

In [ ]:
a = T.scalar('a', dtype='float32')
b = T.scalar('b', dtype='float32')
x = T.scalar('x', dtype='float32')

res = a*x + b

# compile the function for our target device (GPU or CPU)
res_f = theano.function(inputs=[a, b, x], outputs=res)

print res_f(a=2, b=3, x=5)

show_computation_graph(res_f)

13.0


### sigmoid(x)

In [ ]:
x = T.scalar('x', dtype='float32')
sigmoid_x = T.nnet.sigmoid(x)
f_x = theano.function(inputs=[x], outputs=sigmoid_x)

In [ ]:
f_x(1)

In [ ]:
xvals = np.linspace(-6, 6, num=50, dtype=np.float32)
yvals = [f_x(v) for v in xvals]
pl.plot(xvals, yvals)

In [ ]:
show_computation_graph(f_x)

#### Same thing but x is a vector

Using a vector for x lets us compute the sigmoid of a vector of values instead of doing it
one-by-one

In [ ]:
x = T.vector('x', dtype='float32')
sigmoid_x = T.nnet.sigmoid(x)
f_x = theano.function(inputs=[x], outputs=sigmoid_x)
# without a for
yvals = f_x(xvals)
pl.plot(xvals, yvals)

In [ ]:
show_computation_graph(f_x)

## Perceptron implementation

We build a perceptron with a sigmoid activation function. That is

$$o_i = sigmoid(W*x_i + b)$$

Where $sigmoid(x) = \frac{1}{1 + e^{-x}}$

The error is simply ($y_i$ being the expected output value)

$$e_i = (y_i - o_i)^2$$

We train the network with minibatch SGD. That is, at each epoch, we compute the average gradient over a bunch of examples as opposed to a single one.

$$W^{t+1} = W^t - \eta \frac{\partial e_i}{\partial W}$$
$$b^{t+1} = b^t - \eta \frac{\partial e_i}{\partial b}$$

Where $\eta$ is the learning rate.

### Error function derivative

Note that if we add an entry set to 1 at the end of the x vector, we can transform our $Wx + b$ into $W\hat{x} + b$ where $\hat{x} = \left[\begin{matrix}x \\ 1 \end{matrix}\right]$

$$l_i = W\hat{x}$$

$$o_i = sigmoid(l_i)$$

$$e_i = (y_i - o_i)^2$$

We want to compute the derivative of $e_i$ w.r.t $W$. From the [chain rule](http://en.wikipedia.org/wiki/Chain_rule), we have


$$\frac{\partial e_i}{\partial W} = \frac{\partial e_i}{\partial o_i} \frac{\partial o_i}{\partial l_i} \frac{\partial l_i}{\partial W}$$

With
$$\frac{\partial e_i}{\partial o_i} = -2(y_i - o_i)$$
$$\frac{\partial o_i}{\partial l_i} = sigmoid(l_i)(1 - sigmoid(l_i))$$
$$\frac{\partial l_i}{\partial W} = \hat{x}$$

Which gives us

$$\frac{\partial e_i}{\partial W} = -2(y_i - o_i) sigmoid(l_i)(1 - sigmoid(l_i)) \hat{x}$$

Which we use to update our weights at each step of gradient descent :

$$W^{t+1} = W^t - \frac{\partial e_i}{\partial W}$$

In [ ]:
# number of features
X_dim = 2

# weights
W_init = 2 * (np.random.random(X_dim) - 0.5)
W = theano.shared(
    value=W_init.reshape(1, -1),
    name='W',
    borrow=True
)

# bias
b_init = 2 * (np.random.random() - 0.5)
b = theano.shared(
    value = b_init,
    name='b',
)

# data matrix
X = T.matrix('X')
# labels vector
y = T.vector('y')

y_pred = T.nnet.sigmoid(T.dot(W, X.T) + b)

error = T.mean(T.sqr(y - y_pred))

# compilation
y_pred_fn = theano.function(
    inputs=[X],
    outputs=y_pred
)

error_fn = theano.function(
    inputs=[X, y],
    outputs=error
)


In [ ]:
# You can explore the computation graph as follow :
if False:
    show_computation_graph(error)

In [ ]:
# A forward pass through our perceptron
error_fn(X=[[0, 1], [0, 2], [3, 4]], y=[0, 1, 2])

In [ ]:
# For backprop, we need the gradient with regard to our weights and bias
g_W = T.grad(cost=error, wrt=W)
g_b = T.grad(cost=error, wrt=b)

In [ ]:
# We can compute the gradient wrt the weights as follow
g_W_fn = theano.function(
    inputs=[X, y],
    outputs=g_W
)

# Gradient for X=[0, 1] and y=0
g_W_fn(X=[[0, 1]], y=[0])

In [ ]:
# same for bias
g_b_fn = theano.function(
    inputs=[X, y],
    outputs=g_b
)

We can check that theano's computed gradient match the derivation we made by hand.

Note that this seems to run into numerical accuracy issues.

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def my_grad(X, y, W, b):
    o = sigmoid(W.dot(X) + b)
    # The last X is the derivative of o wrt X, which is X
    gW = -2*(y - o)*sigmoid(o)*(1 - sigmoid(o))*X
    # The last 1 is the derivative of o wrt b, which is 1
    gb = -2*(y - o)*sigmoid(o)*(1 - sigmoid(o))*1

    return gW, gb

X1 = np.array([0, 1], dtype=np.float32)
y1 = np.array([0], dtype=np.float32)
print my_grad(X1, y1, W.get_value(), b.get_value())
print g_W_fn([X1], y1), g_b_fn([X1], y1)

print '--'
X1 = np.array([-0.2, 0.7], dtype=np.float32)
y1 = np.array([0.5], dtype=np.float32)
print my_grad(X1, y1, W.get_value(), b.get_value())
print g_W_fn([X1], y1), g_b_fn([X1], y1)

(array([ 0.     ,  0.10959]), array([ 0.10959]))
[[ 0.       0.07662]] 0.0766240492623
--
(array([ 0.02558, -0.08954]), array([-0.12791]))
[[ 0.01896 -0.06636]] -0.0948039833346


We could use g_W_fn to loop over our samples and update our weights. But theano has a nice *updates* parameters that you can give to the *theano.function*. It specifies which variables should be updated and how.

In [ ]:
learning_rate = 1
updates = [
    (W, W - learning_rate * g_W),
    (b, b - learning_rate * g_b)
]

train_model = theano.function(
    inputs=[X, y],
    outputs=error,
    updates=updates
)

predict = theano.function(
    inputs=[X],
    outputs=y_pred
)

In [ ]:
W.set_value([[0.5, 0.5]])
b.set_value(0)

X1 = np.array([[0, 1]], dtype=np.float32)
y1 = np.array([0], dtype=np.float32)
print "prediction        ", predict(X1)
for i in xrange(5):
    print '---- iteration', i
    print "W, b before       ", W.get_value(), b.get_value()
    print "gradient wrt W, b ", g_W_fn(X=X1, y=y1), g_b_fn(X=X1, y=y1)
    train_model(X=X1, y=y1)
    print "W, b after updates", W.get_value(), b.get_value()
    print "prediction        ", predict(X1)

prediction         [[ 0.62246]]
---- iteration 0
W, b before        [[ 0.5  0.5]] 0.0
gradient wrt W, b  [[ 0.       0.29256]] 0.292560507054
W, b after updates [[ 0.5      0.20744]] -0.292560507054
prediction         [[ 0.47873]]
---- iteration 1
W, b before        [[ 0.5      0.20744]] -0.292560507054
gradient wrt W, b  [[ 0.       0.23893]] 0.238933228816
W, b after updates [[ 0.5     -0.03149]] -0.53149373587
prediction         [[ 0.36286]]
---- iteration 2
W, b before        [[ 0.5     -0.03149]] -0.53149373587
gradient wrt W, b  [[ 0.       0.16778]] 0.167778792514
W, b after updates [[ 0.5     -0.19927]] -0.699272528385
prediction         [[ 0.28935]]
---- iteration 3
W, b before        [[ 0.5     -0.19927]] -0.699272528385
gradient wrt W, b  [[ 0.     0.119]] 0.118995825535
W, b after updates [[ 0.5     -0.31827]] -0.818268353919
prediction         [[ 0.24296]]
---- iteration 4
W, b before        [[ 0.5     -0.31827]] -0.818268353919
gradient wrt W, b  [[ 0.       0.08937]] 0.0

### Important note

Note that this is a naive implementation and it will suffer from one major problem if ran onto the GPU : the data (X) will be uploaded to the GPU on every call to our functions. This will most likely using the GPU will be slower than a CPU for this particular implementation.

Of course, Theano allows you to avoid this and upload your data once to the GPU. The [logistic regression theano tutorial](http://deeplearning.net/tutorial/logreg.html) is a good reference for how to do minibatch training with the data in GPU memory. The key is to upload X and y once and then simply provide the index of the sample as an argument to our various functions.

### Let's put everything in a class

This is a perceptron class that contains the same code as above, for ease-of-reuse

In [ ]:
class Perceptron(object):
    def __init__(self, X_dim, learning_rate=0.2):
        # theano variable definitions
        W_init = (2 * np.random.random(X_dim)) * 0.25
        self.W = theano.shared(
            value=W_init.reshape(1, -1),
            name='W',
            borrow=True
        )
        b_init = (2 * np.random.random()) * 0.25
        self.b = theano.shared(
            value = b_init,
            name='b',
        )

        self.X = T.matrix('X')
        self.y = T.vector('y')

        self.y_pred = T.nnet.sigmoid(T.dot(self.W, self.X.T) + self.b)

        self.error = T.mean(T.sqr(self.y - self.y_pred))

        self.g_W = T.grad(cost=self.error, wrt=self.W)
        self.g_b = T.grad(cost=self.error, wrt=self.b)

        # theano functions
        self.predict_fn = theano.function(
            inputs=[self.X],
            outputs=self.y_pred
        )

        self.error_fn = theano.function(
            inputs=[self.X, self.y],
            outputs=self.error
        )

        updates = [
            (self.W, self.W - learning_rate * self.g_W),
            (self.b, self.b - learning_rate * self.g_b)
        ]

        self.train_model_fn = theano.function(
            inputs=[self.X, self.y],
            outputs=self.error,
            updates=updates
        )

        # non-theano stuff
        self.train_errors = []
        self.validation_errors = []

    def train_gd(self, X_train, y_train, X_validation, y_validation, epochs=10):
        """Simple gradient descent training"""
        assert len(X_train.shape) == 2, "X must be 2D"

        for epoch in range(epochs):
            # This does forward propagation, gradient computation and weights
            # update all in one
            self.train_model_fn(X_train, y_train)

            # evaluate train/validation errors
            self.train_errors.append(
                self.error_fn(X_train, y_train)
            )
            self.validation_errors.append(
                self.error_fn(X_validation, y_validation)
            )

        return self

    def predict(self, X):
        """Binary (0, 1) prediction"""
        return self.predict_fn(X).astype(np.int)

    def decision_function(self, X):
        """Float [0,1] prediction"""
        return self.predict_fn(X)

## Generate a toy dataset and test

In [ ]:
def dataset_fixed_cov():
    """Generate 2 Gaussians samples with the same covariance matrix"""
    n, dim = 300, 2
    np.random.seed(42)
    C = np.array([[0., -0.23], [0.83, .23]])
    X = np.r_[np.dot(np.random.randn(n, dim), C),
              np.dot(np.random.randn(n, dim), C) + np.array([1, 1])]
    y = np.hstack((np.zeros(n), np.ones(n))).astype(np.int)
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    return X, y

X, y = dataset_fixed_cov()

print X

[[-0.11476 -0.14605]
 [ 1.26411  0.20133]
 [-0.19433  0.     ]
 ..., 
 [ 1.36559  1.4677 ]
 [ 1.45857  1.13159]
 [ 2.13224  1.26225]]


In [ ]:
pl.scatter(X[y==0, 0], X[y==0, 1], c='r')
pl.scatter(X[y==1, 0], X[y==1, 1], c='b')

In [ ]:
X_train, X_validation, y_train, y_validation = skcross.train_test_split(X, y, test_size=0.33, random_state=20)
perceptron = Perceptron(X.shape[1], learning_rate=0.2)
perceptron.train_gd(X_train, y_train, X_validation, y_validation, epochs=800)
#perceptron.train_minisgd(X_train, y_train, X_validation, y_validation, epochs=500, minibatch_size=10)

In [ ]:
pl.plot(perceptron.train_errors, label='train')
pl.plot(perceptron.validation_errors, label='validation')
pl.legend()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec

def show_decision_boundary(clf, X, y, subplot_spec=None):
    """
    Utility function to plot the decision function of a classifier
    """
    assert X.shape[1] == 2
    wratio = (15, 1)
    if subplot_spec is None:
        gs = gridspec.GridSpec(1,2, width_ratios=wratio)
    else:
        gs = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=subplot_spec, width_ratios=wratio)

    ax = pl.subplot(gs[0])
    ax.set_title('Dataset and decision function')

    x_min, x_max = X[:,0].min() - 1, X[:,0].max() + 1
    y_min, y_max = X[:,1].min() - 1, X[:,1].max() + 1
    h = 0.2 # step size in the meshgrid
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    xx = xx.astype(np.float32)
    yy = yy.astype(np.float32)

    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ctr = ax.contourf(xx, yy, Z, cmap=cm.gray, vmin=0, vmax=1)

    unique_labels = np.unique(y)
    colors = cm.Paired(np.linspace(0, 1, num=len(unique_labels)))
    for i, yi in enumerate(unique_labels):
        color = colors[i]
        ax.scatter(X[y == yi, 0], X[y == yi, 1], c=color, linewidth=0, label='%d' % yi)
    ax.legend()
    ax.set_xlim((x_min, x_max))
    ax.set_ylim((y_min, y_max))

    pl.colorbar(ctr, cax=pl.subplot(gs[1]))

show_decision_boundary(perceptron, X, y)

## Making a video of the training


In [ ]:
import matplotlib.animation as manimation
from tempfile import NamedTemporaryFile
from matplotlib.legend_handler import HandlerLine2D

# Depending on your environment, you might have to use any of the 2 following
AVConvWriter = manimation.writers['ffmpeg'] # or 'avconv' if you're on ubuntu
metadata = dict(title='Perceptron training', artist='',comment='')
writer = AVConvWriter(fps=4, metadata=metadata, codec="libx264", extra_args=['-vcodec', 'libx264'])

#writer = manimation.FFMpegFileWriter(fps=4)

perceptron = Perceptron(X.shape[1])

fig = pl.figure(figsize=(10, 5))
gs = gridspec.GridSpec(1, 2, wspace=0.4)

nepochs = 800

err_ymax = None

with NamedTemporaryFile(suffix='.mp4') as f:
    with writer.saving(fig, f.name, 100):
        for epoch in xrange(nepochs):
            perceptron.train_gd(X_train, y_train, X_validation, y_validation, epochs=1)
            #perceptron.train_minisgd(X_train, y_train, X_validation, y_validation, epochs=1, minibatch_size=10)

            if err_ymax is None:
                err_ymax = max(np.max(perceptron.train_errors),
                               np.max(perceptron.validation_errors)) * 1.1

            if epoch % 10 == 0: # Capturing all 800 epochs takes forever...
                show_decision_boundary(perceptron, X, y, gs[0])

                ax = pl.subplot(gs[1])
                pl.title('Error')
                line_train, = ax.plot(np.array(perceptron.train_errors), c='b', label='train')
                line_valid, = ax.plot(np.array(perceptron.validation_errors), c='g', label='validation')
                ax.set_xlim(0, nepochs)
                ax.set_ylim(0, err_ymax)
                ax.set_xlabel('epochs')
                ax.set_ylabel('error')
                ax.legend((line_train, line_valid), ('train', 'validation'))

                writer.grab_frame()

    video = open(f.name, "rb").read()

In [ ]:
from IPython.display import HTML

VIDEO_TAG = """<video style="width:80%" controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

HTML(data=VIDEO_TAG.format(video.encode("base64")))

## Interesting links

Tutorials :

[Theano tutorials](http://deeplearning.net/software/theano/tutorial/)

[Deep learning tutorials using Theano](http://deeplearning.net/tutorial/contents.html)

Deep learning libraries using theano :

[Lasagne](https://github.com/Lasagne/Lasagne)

[Blocks](https://github.com/bartvm/blocks)

[pylearn2](http://deeplearning.net/software/pylearn2/)

General deep learning :

[Reading recommendations](http://deeplearning.net/reading-list/)

[Software](http://deeplearning.net/software_links/)

[Stanford course about CNN](http://cs231n.stanford.edu/)

Cool demos :

[CNN in your browser](http://cs.stanford.edu/people/karpathy/convnetjs/)

Misc :

[How to access theano generated code](https://groups.google.com/forum/#!topic/theano-users/o-BLhLvnL5s)



## Additional stuff

To add minibatch SGD training to the perceptron, use the following code. Note that since we do more weight updates per epoch (since we update for each minibatch), this requires less epoch to converge.

    def train_minisgd(self, X_train, y_train, X_validation, y_validation, epochs=10, minibatch_size=10):
        """Training with minibatch SGD"""
        assert len(X_train.shape) == 2, "X must be 2D"
        
        for epoch in range(epochs):
            minibatch_indices = np.arange(X_train.shape[0])
            np.random.shuffle(minibatch_indices)
            # for each minibatch, compute gradient of weights
            for start in xrange(0, len(minibatch_indices), minibatch_size):
                end = start + minibatch_size
                indices = minibatch_indices[start:end]
                Xb = X_train[indices]
                yb = y_train[indices]
                
                self.train_model_fn(Xb, yb)
            
            # evaluate train/validation errors
            self.train_errors.append(
                self.error_fn(X_train, y_train)
            )
            self.validation_errors.append(
                self.error_fn(X_validation, y_validation)
            )
        
        return self